# Create IoT Data
This Python code is copied from Jules S. Damji's public github page https://github.com/dmatrix/examples

The file can be found [here](https://github.com/dmatrix/examples/blob/master/spark/databricks/notebooks/py/sql_device_provisioning.ipynb).

Added a few lines to write a json file containing the iot data.

*Christoph Windheuser, April 11, 2022*

In [36]:
#import the necessary modules
import time
import random
import json
import time
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

def getLargeBatches(size):
    batches = []
    for i in range(1, size):
        batches.append(str(i))
    return batches

#
# get an IP
#

def get_ip_addr():
    n1 = random.randrange(60, 209)
    n2 = random.randrange(1, 127)
    n3 = random.randrange(1, 127)
    n4 = random.randrange(1,127)

    ip = str(n1) + "." + str(n2) + "." + str (n3) + "." + str(n4)
    return ip

#
# get random letters
#

def get_random_word():
    word = ''
    for i in range(10):
        word += random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789')
    return word

#
# create a json object with attributes and values
#

def create_json(id, d):
    print (id, d)
    temp = random.randrange(10, 35)
    (x , y) = random.randrange(0, 100), random.randrange(0, 100)
    ts = str(time.time())
    ip = get_ip_addr()
    if id % 2 == 0:
        d = "sensor-pad-" + d + get_random_word()
    elif id % 3 == 0:
        d = "device-mac-" + d + get_random_word()
    elif id % 5 == 0:
        d = "therm-stick-" + d + get_random_word()
    else:
        d = "meter-gauge-" + d + get_random_word()
    zipcode = random.randrange(94538,97107)
    humidity = random.randrange(25, 100)
    return json.dumps({"device_id": id, "device_name": d, "ip": ip, "timestamp": ts, "temp": temp,
                       "scale": "Celsius", "lat": x, "long": y, "zipcode": zipcode, "humidity": humidity}, 
                        sort_keys=True)

#
#
# create a list of devices 
#

def create_devices(batches):
    id=0
    devices = []
    for b in batches:
        for w in b:
            id=id+1
            device_msg = create_json(id, w)
            devices.append(device_msg)
    return devices


In [37]:
batches = getLargeBatches(5)
print (batches)

devices = create_devices(batches)
print (devices)

['1', '2', '3', '4']
1 1
2 2
3 3
4 4
['{"device_id": 1, "device_name": "meter-gauge-11mIfS4oiK3", "humidity": 62, "ip": "120.60.24.117", "lat": 58, "long": 95, "scale": "Celsius", "temp": 10, "timestamp": "1649700077.7724078", "zipcode": 94586}', '{"device_id": 2, "device_name": "sensor-pad-2HZu1JHDoaK", "humidity": 60, "ip": "148.46.41.6", "lat": 35, "long": 97, "scale": "Celsius", "temp": 12, "timestamp": "1649700077.7726402", "zipcode": 94687}', '{"device_id": 3, "device_name": "device-mac-3ME1sUSu3dX", "humidity": 47, "ip": "123.31.70.75", "lat": 35, "long": 81, "scale": "Celsius", "temp": 24, "timestamp": "1649700077.7728536", "zipcode": 97083}', '{"device_id": 4, "device_name": "sensor-pad-47Z7MqT6Gva", "humidity": 35, "ip": "70.68.100.34", "lat": 15, "long": 46, "scale": "Celsius", "temp": 14, "timestamp": "1649700077.7730544", "zipcode": 94942}']


In [23]:
# Write json to file
with open('data/iot_devices.json', 'w') as f:
    for d in devices:
        f.write(d  + '\n')
